In [17]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics


In [18]:
raw_env_data = pd.read_csv("raw_env_data.csv")
raw_temp = raw_env_data.iloc[:,0].values.reshape(-1, 1)
raw_humi = raw_env_data.iloc[:,1].values.reshape(-1, 1)
env_temp = raw_env_data.iloc[:,2].values.reshape(-1, 1)
env_humi = raw_env_data.iloc[:,3].values.reshape(-1, 1)
X = np.concatenate((raw_temp, raw_humi), axis = 1)

def train_raw_to_env_model(X,Y1,Y2):
    lm_temp = LinearRegression()
    lm_humi = LinearRegression()
    lm_temp.fit(X, Y1)
    lm_humi.fit(X, Y2)
    pred_env_temp = lm_temp.predict(X)
    pred_env_humi = lm_humi.predict(X)
    temp_acc = lm_temp.score(X, Y1)
    humi_acc = lm_humi.score(X, Y2)
    print("env_temp accuracy:", temp_acc)
    print("env_humi accuracy:", humi_acc)
    joblib.dump(lm_temp,"./Predict_env_temp.pkl")
    joblib.dump(lm_temp,"./Predict_env_humi.pkl")

train_raw_to_env_model(X,env_temp,env_humi)


env_temp accuracy: 0.9625985904742477
env_humi accuracy: 0.9761670919599814


In [19]:
weather_data = pd.read_csv("weather_data_1hr.csv")
temp = weather_data.iloc[:,5].values.reshape(-1, 1)
humi = weather_data.iloc[:,15].values.reshape(-1, 1)
press = weather_data.iloc[:,18].values.reshape(-1, 1)
X = np.concatenate((temp, humi, press), axis = 1)
Y = weather_data.iloc[:,30].values.reshape(-1, 1)

def train_env_to_heat_index_model(X,Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.01, random_state=0)
    lm = LinearRegression()
    lm.fit(X_train, Y_train)
    Y_pred_train = lm.predict(X_train)
    Y_pred_test = lm.predict(X_test)
    train_acc = lm.score(X_train, Y_train)
    test_acc = lm.score(X_test, Y_test)
    print("Train accuracy:", train_acc)
    print("Test accuracy:", test_acc)
    joblib.dump(lm,"./Predict_Heat_Injury.pkl")
    
train_env_to_heat_index_model(X, Y)


Train accuracy: 0.9703356707892048
Test accuracy: 0.968950458787886


In [43]:
def convert_raw_into_env(raw_temp,raw_humi):
    model_temp=joblib.load(filename="Predict_env_temp.pkl")
    model_humi=joblib.load(filename="Predict_env_humi.pkl")
    env_temp=model_temp.predict(np.concatenate((np.array(raw_temp).reshape(-1, 1),np.array(raw_humi).reshape(-1, 1)), axis = 1))
    env_humi=model_humi.predict(np.concatenate((np.array(raw_temp).reshape(-1, 1),np.array(raw_humi).reshape(-1, 1)), axis = 1))
    return env_temp, env_humi

def predict_heat_index(tempC,humi,press):
    
    tempF=tempC*1.8+32
    model=joblib.load(filename="Predict_Heat_Injury.pkl")
    feelLike=model.predict(np.concatenate((np.array(tempF).reshape(-1, 1),np.array(humi).reshape(-1, 1) , np.array(press).reshape(-1, 1)),axis = 1))
    print("feels like:",(feelLike[0][0]-32)/1.8)
    return (feelLike[0][0]-32)/1.8

def predict_heat_injury_risk_level(raw_tempC,raw_humi,press):
    env_tempC, env_humi=convert_raw_into_env(raw_tempC,raw_humi)
    feelLike=predict_heat_index(env_tempC,env_humi,press)
    if (feelLike>52.2): return 4 #extremely dangerous
    if (feelLike>39.4): return 3 #dangerous
    if (feelLike>32.8): return 2 #warning
    if (feelLike>26.7): return 1 #attention 
    return 0 # safe


In [44]:
level=["safe","attention","warning","dangerous","extremely dangerous"]
print("risk level:",level[predict_heat_injury_risk_level(36,90,1000)])


feels like: 39.99621838447035
risk level: dangerous
